In [5]:
import pandas as pd

# ✅ Load CSV (using forward slashes so no escape sequence issues)

# Load datasets
matches = pd.read_csv("IPL Dataset/matches.csv")
deliveries = pd.read_csv("IPL Dataset/deliveries.csv")

# ✅ Make sure all columns show in one line (no wrapping with \)
pd.set_option("display.max_columns", 600)
pd.set_option("display.width", 600)

matches.head(1)  # Display the first few rows of the DataFrame


,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


Player won MOM without winning team


In [ ]:

# Step 1: Take match_id, winner, player_of_match
mom_df = matches[['id','Season','winner','player_of_match']].rename(columns={'Season':'Season'})

# Step 2: From deliveries, create a mapping of player -> team (per match)
# For batsmen
batsman_team = deliveries.groupby(['match_id','batsman'])['batting_team'].first().reset_index()
batsman_team.rename(columns={'batsman':'player','batting_team':'team'}, inplace=True)

# For bowlers
bowler_team = deliveries.groupby(['match_id','bowler'])['bowling_team'].first().reset_index()
bowler_team.rename(columns={'bowler':'player','bowling_team':'team'}, inplace=True)

# Combine both (some PoM are bowlers, some batsmen)
player_team_map = pd.concat([batsman_team, bowler_team]).drop_duplicates()

# Step 3: Merge PoM info with their team
mom_with_team = mom_df.merge(player_team_map, left_on=['id','player_of_match'], right_on=['match_id','player'], how='left')

# Step 4: Filter where player's team != winner (MoM from losing side)
losing_side_mom = mom_with_team[mom_with_team['team'] != mom_with_team['winner']]

# Step 5: Show results with Season (capitalized)
print(losing_side_mom[['Season','player_of_match','team','winner']].tail(5))


In [ ]:
matches.query("city == 'Mumbai'")

101

In [ ]:
toss =matches['toss_winner'].value_counts()


In [31]:
toss = matches.query("toss_winner == winner")
display(toss[["Season", "toss_winner", "winner"]].head(5))


,Season,toss_winner,winner
1,IPL-2017,Rising Pune Supergiant,Rising Pune Supergiant
2,IPL-2017,Kolkata Knight Riders,Kolkata Knight Riders
3,IPL-2017,Kings XI Punjab,Kings XI Punjab
4,IPL-2017,Royal Challengers Bangalore,Royal Challengers Bangalore
5,IPL-2017,Sunrisers Hyderabad,Sunrisers Hyderabad


In [ ]:
matches.query("result == 'tie'")

In [ ]:
matches.query("((team1 == 'Mumbai Indians') or (team2 == 'Mumbai Indians')) and (winner != 'Mumbai Indians')")


In [15]:
matches.query("winner == 'Rajasthan Royals' and Season == 'IPL-2008' and (team1 == 'Chennai Super Kings' or team2 == 'Chennai Super Kings')")



,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
82,83,IPL-2008,Jaipur,04-05-2008,Chennai Super Kings,Rajasthan Royals,Chennai Super Kings,bat,normal,0,Rajasthan Royals,0,8,Sohail Tanvir,Sawai Mansingh Stadium,Asad Rauf,AV Jayaprakash,NaN
109,110,IPL-2008,Chennai,24-05-2008,Rajasthan Royals,Chennai Super Kings,Rajasthan Royals,bat,normal,0,Rajasthan Royals,10,0,JA Morkel,"MA Chidambaram Stadium, Chepauk",DJ Harper,SL Shastri,NaN
116,117,IPL-2008,Mumbai,01-06-2008,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,normal,0,Rajasthan Royals,0,3,YK Pathan,Dr DY Patil Sports Academy,BF Bowden,RE Koertzen,NaN


In [ ]:
# Correct way
# Correct way
csk_matches = matches.query("((`team1` == 'Chennai Super Kings') or (`team2` == 'Chennai Super Kings')) and winner != 'Chennai Super Kings' and Season == 'IPL-2008'")
print(csk_matches.shape[0])



Find the percentage of times the toss winner chose to bat

In [28]:
bat_count = matches.query("toss_decision == 'bat'").shape[0]
total_tosses = matches.shape[0]
print(f"Percentage: {(bat_count / total_tosses) * 100:.2f}%")


Percentage: 38.76%
